# Fine-tune BERT for Token Classification

In [ ]:
### WARNING: Do NOT run this cell, unless you are running this on Google Colab. For a local installation run: pip install -r requirements.txt inside the terminal
# % pip install transformers==4.9.1
# % pip install datasets==1.11.0
# % pip install tabulate==0.8.9
# % pip install seqeval==1.2.2

In [1]:
import random, time, os
import torch
from torch.nn import CrossEntropyLoss
import numpy as np
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from transformers import BertForTokenClassification, AdamW
from transformers import get_linear_schedule_with_warmup
import logging, sys
from transformers import BertTokenizer
import pandas as pd

# Our code behind the scenes!
import bert_utils_srl as utils
import make_jsonl as jsonl

/opt/anaconda3/envs/bert4ner/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-08 17:30:04.239415: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Read dataset and convert to jsonl format

In [2]:
# get datafiles to jsonl format:
train_jsonl_file, test_jsonl_file = jsonl.main(['make_jsonl.py', False, True])

/Users/cece/Desktop/github/AdvancedNLP-Group3/Assignment 3


## Initialize Hyperparameters

In [3]:
EPOCHS = 2

### ------ CHANGE MODEL NAME to bert-base-multilingual-cased ------ ###
BERT_MODEL_NAME = "bert-base-multilingual-cased"
GPU_RUN_IX=0

SEED_VAL = 1234500
SEQ_MAX_LEN = 512 #256
PRINT_INFO_EVERY = 10 # Print status only every X batches
GRADIENT_CLIP = 1.0
LEARNING_RATE = 1e-5
BATCH_SIZE = 4

### ------ update training data jsonl file ------ ###
TRAIN_DATA_PATH = train_jsonl_file #"data/trial_mini_data.conll" # "data/conll2003.train.conll"

# IMPORTANT NOTE: We use as validation set the test portion, in order to avoid overfitting on the dev set, 
# and this way be able to evaluate later and compare with your previous models!

### ------ update testing data jsonl file ------ ###
DEV_DATA_PATH =  test_jsonl_file #"data/trial_mini_data.conll" # "data/conll2003.dev.conll"

SAVE_MODEL_DIR = "saved_models/MY_BERT_NER/"

LABELS_FILENAME = f"{SAVE_MODEL_DIR}/label2index.json"
PRED_FILENAME = f"{SAVE_MODEL_DIR}/pred2index.json"

LOSS_TRN_FILENAME = f"{SAVE_MODEL_DIR}/Losses_Train_{EPOCHS}.json"
LOSS_DEV_FILENAME = f"{SAVE_MODEL_DIR}/Losses_Dev_{EPOCHS}.json"

PAD_TOKEN_LABEL_ID = CrossEntropyLoss().ignore_index # -100

if not os.path.exists(SAVE_MODEL_DIR):
    os.makedirs(SAVE_MODEL_DIR)


# Initialize Random seeds and validate if there's a GPU available...
device, USE_CUDA = utils.get_torch_device(GPU_RUN_IX)
random.seed(SEED_VAL)
np.random.seed(SEED_VAL)
torch.manual_seed(SEED_VAL)
torch.cuda.manual_seed_all(SEED_VAL)

## Record everything inside a Log File

In [4]:
console_hdlr = logging.StreamHandler(sys.stdout)
file_hdlr = logging.FileHandler(filename=f"{SAVE_MODEL_DIR}/BERT_TokenClassifier_train_{EPOCHS}.log")
logging.basicConfig(level=logging.INFO, handlers=[console_hdlr, file_hdlr])
logging.info("Start Logging")

INFO:root:Start Logging


## Load Training and Validation Datasets

In [6]:
# Initialize Tokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_NAME, do_basic_tokenize=False)

# Load Train Dataset
train_data, train_labels, train_label2index, train_pred_sense, train_pred2index = utils.read_json_srl(TRAIN_DATA_PATH) #, has_labels=True)
train_inputs, train_masks, train_labels, seq_lengths, train_pred = utils.data_to_tensors(train_data, 
                                                                            tokenizer, 
                                                                            max_len=SEQ_MAX_LEN, 
                                                                            labels=train_labels, 
                                                                            label2index=train_label2index,
                                                                            pred_sense=train_pred_sense,
                                                                            pred2index=train_pred2index,
                                                                            pad_token_label_id=PAD_TOKEN_LABEL_ID)


utils.save_label_dict(train_label2index, filename=LABELS_FILENAME)
index2label = {v: k for k, v in train_label2index.items()}     

INFO:bert_utils_srl:Read 1 Sentences!
INFO:bert_utils_srl:Read 2 Sentences!
INFO:bert_utils_srl:Read 3 Sentences!
INFO:bert_utils_srl:Read 4 Sentences!
INFO:bert_utils_srl:Read 5 Sentences!
INFO:bert_utils_srl:Read 6 Sentences!
INFO:bert_utils_srl:Read 7 Sentences!
INFO:bert_utils_srl:Read 8 Sentences!
INFO:bert_utils_srl:Read 9 Sentences!
INFO:bert_utils_srl:Read 10 Sentences!
INFO:bert_utils_srl:Read 11 Sentences!
INFO:bert_utils_srl:Read 12 Sentences!
INFO:bert_utils_srl:Read 13 Sentences!
INFO:bert_utils_srl:Read 14 Sentences!
INFO:bert_utils_srl:Read 15 Sentences!
INFO:bert_utils_srl:Read 16 Sentences!
INFO:bert_utils_srl:Read 17 Sentences!
INFO:bert_utils_srl:Read 18 Sentences!
INFO:bert_utils_srl:Read 19 Sentences!
INFO:bert_utils_srl:Read 20 Sentences!
INFO:bert_utils_srl:Read 21 Sentences!
INFO:bert_utils_srl:Read 22 Sentences!
INFO:bert_utils_srl:Read 23 Sentences!
INFO:bert_utils_srl:Read 24 Sentences!
INFO:bert_utils_srl:Read 25 Sentences!
INFO:bert_utils_srl:Read 26 Senten

In [7]:
# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs, train_masks, train_labels, train_pred)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=BATCH_SIZE)

In [9]:
# Load Dev Dataset
dev_data, dev_labels, _ , dev_pred_sense, _ = utils.read_json_srl(DEV_DATA_PATH) #, has_labels=True)
dev_inputs, dev_masks, dev_labels, dev_lens, dev_pred = utils.data_to_tensors(dev_data, 
                                                                    tokenizer, 
                                                                    max_len=SEQ_MAX_LEN, 
                                                                    labels=dev_labels, 
                                                                    label2index=train_label2index,
                                                                    pred_sense=dev_pred_sense,
                                                                    pred2index=train_pred2index,
                                                                    pad_token_label_id=PAD_TOKEN_LABEL_ID)  

# Create the DataLoader for our Development set.
dev_data = TensorDataset(dev_inputs, dev_masks, dev_labels, dev_pred) 
dev_sampler = RandomSampler(dev_data)     
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=BATCH_SIZE)

INFO:bert_utils_srl:Read 1 Sentences!
INFO:bert_utils_srl:Read 2 Sentences!
INFO:bert_utils_srl:Read 3 Sentences!
INFO:bert_utils_srl:Read 4 Sentences!
INFO:bert_utils_srl:Read 5 Sentences!
INFO:bert_utils_srl:Read 6 Sentences!
INFO:bert_utils_srl:Read 7 Sentences!
INFO:bert_utils_srl:Read 8 Sentences!
INFO:bert_utils_srl:Read 9 Sentences!
INFO:bert_utils_srl:Read 10 Sentences!
INFO:bert_utils_srl:Read 11 Sentences!
INFO:bert_utils_srl:Read 12 Sentences!
INFO:bert_utils_srl:Read 13 Sentences!
INFO:bert_utils_srl:Read 14 Sentences!
INFO:bert_utils_srl:Read 15 Sentences!
INFO:bert_utils_srl:Read 16 Sentences!
INFO:bert_utils_srl:Read 17 Sentences!
INFO:bert_utils_srl:MAX TOKENIZED SEQ LENGTH IN DATASET IS 43


## Initialize Model Components

In [10]:
model = BertForTokenClassification.from_pretrained(BERT_MODEL_NAME, num_labels=len(train_label2index))
model.config.finetuning_task = 'token-classification'
model.config.id2label = index2label  
model.config.label2id = train_label2index  
if USE_CUDA: model.cuda()

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * EPOCHS

# Create optimizer and the learning rate scheduler.
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps=0,
                                        num_training_steps=total_steps)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

## Training Cycle (Fine-tunning)

In [11]:
loss_trn_values, loss_dev_values = [], []

for epoch_i in range(1, EPOCHS+1):
    # Perform one full pass over the training set.
    logging.info("")
    logging.info('======== Epoch {:} / {:} ========'.format(epoch_i, EPOCHS))
    logging.info('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    # For each batch of training data...
    print("start processing train_dataloader ......")
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        b_preds = batch[3].to(device) ###### added predicate ######

        model.zero_grad()

        # Perform a forward pass (evaluate the model on this training batch).
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels, token_type_ids=b_preds) #input_ids = 
        ### added token_tupe_ids = b_preds, also 'input_ids')
        loss = outputs[0]
        total_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)

        # Update parameters
        optimizer.step()
        scheduler.step()

        # Progress update
        if step % PRINT_INFO_EVERY == 0 and step != 0:
            # Calculate elapsed time in minutes.
            elapsed = utils.format_time(time.time() - t0)
            # Report progress.
            logging.info('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.    Loss: {}.'.format(step, len(train_dataloader),
                                                                                            elapsed, loss.item()))

    # Calculate the average loss over the training data.
    print("calculating the avg loss ......")
    avg_train_loss = total_loss / len(train_dataloader)

    # Store the loss value for plotting the learning curve.
    print("log the loss value  ......")
    loss_trn_values.append(avg_train_loss)

    logging.info("")
    logging.info("  Average training loss: {0:.4f}".format(avg_train_loss))
    logging.info("  Training Epoch took: {:}".format(utils.format_time(time.time() - t0)))

    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on our validation set.
    print("starts validation ......")
    t0 = time.time()
    results, preds_list = utils.evaluate_bert_model(dev_dataloader, BATCH_SIZE, model, tokenizer, index2label, PAD_TOKEN_LABEL_ID, prefix="Validation Set")
    
    loss_dev_values.append(results['loss'])
    logging.info("  Validation Loss: {0:.2f}".format(results['loss']))
    logging.info("  Precision: {0:.2f} || Recall: {1:.2f} || F1: {2:.2f}".format(results['precision']*100, results['recall']*100, results['f1']*100))
    logging.info("  Validation took: {:}".format(utils.format_time(time.time() - t0)))


    # Save Checkpoint for this Epoch
    utils.save_model(f"{SAVE_MODEL_DIR}/EPOCH_{epoch_i}", {"args":[]}, model, tokenizer)


utils.save_losses(loss_trn_values, filename=LOSS_TRN_FILENAME)
utils.save_losses(loss_dev_values, filename=LOSS_DEV_FILENAME)
logging.info("")
logging.info("Training complete!")

INFO:root:
INFO:root:======== Epoch 1 / 2 ========
INFO:root:Training...
start processing train_dataloader ......
INFO:root:  Batch    10  of     25.    Elapsed: 0:03:14.    Loss: 0.6744903326034546.
INFO:root:  Batch    20  of     25.    Elapsed: 0:05:56.    Loss: 0.5470535159111023.
calculating the avg loss ......
log the loss value  ......
INFO:root:
INFO:root:  Average training loss: 1.1210
INFO:root:  Training Epoch took: 0:07:02
starts validation ......
INFO:bert_utils_srl:***** Running evaluation Validation Set *****
INFO:bert_utils_srl:  Batch size = 4
INFO:bert_utils_srl:{0: 'O', 1: 'B-ARG0', 2: 'B-V', 3: 'B-ARG1', 4: 'B-ARGM-LOC', 5: 'B-ARGM-MOD', 6: 'B-ARGM-GOL', 7: 'B-ARGM-TMP', 8: 'B-ARG2', 9: 'B-ARGM-ADV', 10: 'B-ARGM-MNR', 11: 'B-ARGM-LVB', 12: 'B-ARGM-PRR', 13: 'B-ARG4', 14: 'B-ARGM-NEG', 15: 'B-ARGM-EXT', 16: 'B-R-ARG0', 17: 'B-ARGM-ADJ', 18: 'B-R-ARG1', 19: 'B-C-V', 20: 'B-ARG3', 21: 'B-ARGM-COM', 22: 'B-ARGM-DIS', 23: 'B-ARGM-CAU'}
INFO:root:  Validation Loss: 0.63
I

/opt/anaconda3/envs/bert4ner/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


INFO:root:
INFO:root:======== Epoch 2 / 2 ========
INFO:root:Training...
start processing train_dataloader ......
INFO:root:  Batch    10  of     25.    Elapsed: 0:02:58.    Loss: 0.502220094203949.
INFO:root:  Batch    20  of     25.    Elapsed: 0:05:40.    Loss: 0.3539716601371765.
calculating the avg loss ......
log the loss value  ......
INFO:root:
INFO:root:  Average training loss: 0.5756
INFO:root:  Training Epoch took: 0:06:49
starts validation ......
INFO:bert_utils_srl:***** Running evaluation Validation Set *****
INFO:bert_utils_srl:  Batch size = 4
INFO:bert_utils_srl:{0: 'O', 1: 'B-ARG0', 2: 'B-V', 3: 'B-ARG1', 4: 'B-ARGM-LOC', 5: 'B-ARGM-MOD', 6: 'B-ARGM-GOL', 7: 'B-ARGM-TMP', 8: 'B-ARG2', 9: 'B-ARGM-ADV', 10: 'B-ARGM-MNR', 11: 'B-ARGM-LVB', 12: 'B-ARGM-PRR', 13: 'B-ARG4', 14: 'B-ARGM-NEG', 15: 'B-ARGM-EXT', 16: 'B-R-ARG0', 17: 'B-ARGM-ADJ', 18: 'B-R-ARG1', 19: 'B-C-V', 20: 'B-ARG3', 21: 'B-ARGM-COM', 22: 'B-ARGM-DIS', 23: 'B-ARGM-CAU'}
INFO:root:  Validation Loss: 0.69
IN

-------------------------------------------------------------------------
# SCRIPT NOT UPDATED BEYOND THIS CELL
-------------------------------------------------------------------------

# Use Fine-tuned Model for Predictions

In [12]:
from transformers import pipeline
from torch.utils.data import SequentialSampler

GPU_IX=0
device, USE_CUDA = utils.get_torch_device(GPU_IX)
FILE_HAS_GOLD = True
SEQ_MAX_LEN = 256
BATCH_SIZE = 4
# IMPORTANT NOTE: We predict on the dev set to make the results comparable with your previous models from this course
TEST_DATA_PATH = test_jsonl_file #"data/trial_mini_data.conll" # "data/conll2003.dev.conll"
# TEST_DATA_PATH = "data/trial_unk_data.conll"
MODEL_DIR = "saved_models/MY_BERT_NER/"
LOAD_EPOCH = 1
INPUTS_PATH=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/model_inputs.txt"
OUTPUTS_PATH=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/model_outputs.txt"
PAD_TOKEN_LABEL_ID = CrossEntropyLoss().ignore_index # -100

console_hdlr = logging.StreamHandler(sys.stdout)
file_hdlr = logging.FileHandler(filename=f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}/BERT_TokenClassifier_predictions.log")
logging.basicConfig(level=logging.INFO, handlers=[console_hdlr, file_hdlr])

## Load Pre-trained Model

In [13]:
model, tokenizer = utils.load_model(BertForTokenClassification, BertTokenizer, f"{MODEL_DIR}/EPOCH_{LOAD_EPOCH}")
label2index = utils.load_label_dict(f"{MODEL_DIR}/label2index.json")
index2label = {v:k for k,v in label2index.items()}

## Load File for Predictions 

In [15]:
test_data, test_labels, _, test_pred_sense, _ = utils.read_json_srl(TEST_DATA_PATH)# , has_labels=FILE_HAS_GOLD)
prediction_inputs, prediction_masks, gold_labels, seq_lens, gold_pred = utils.data_to_tensors(test_data, 
                                                                                   tokenizer, 
                                                                                   max_len=SEQ_MAX_LEN, 
                                                                                   labels=test_labels, 
                                                                                   label2index=label2index,
                                                                                   pred2index=train_pred2index,
                                                                                   pred_sense=test_pred_sense)


INFO:bert_utils_srl:Read 1 Sentences!
INFO:bert_utils_srl:Read 2 Sentences!
INFO:bert_utils_srl:Read 3 Sentences!
INFO:bert_utils_srl:Read 4 Sentences!
INFO:bert_utils_srl:Read 5 Sentences!
INFO:bert_utils_srl:Read 6 Sentences!
INFO:bert_utils_srl:Read 7 Sentences!
INFO:bert_utils_srl:Read 8 Sentences!
INFO:bert_utils_srl:Read 9 Sentences!
INFO:bert_utils_srl:Read 10 Sentences!
INFO:bert_utils_srl:Read 11 Sentences!
INFO:bert_utils_srl:Read 12 Sentences!
INFO:bert_utils_srl:Read 13 Sentences!
INFO:bert_utils_srl:Read 14 Sentences!
INFO:bert_utils_srl:Read 15 Sentences!
INFO:bert_utils_srl:Read 16 Sentences!
INFO:bert_utils_srl:Read 17 Sentences!
INFO:bert_utils_srl:MAX TOKENIZED SEQ LENGTH IN DATASET IS 43


## Make Predictions

In [16]:
if FILE_HAS_GOLD:
    prediction_data = TensorDataset(prediction_inputs, prediction_masks, gold_labels, gold_pred)
    prediction_sampler = SequentialSampler(prediction_data)
    prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=BATCH_SIZE)

    logging.info('Predicting labels for {:,} test sentences...'.format(len(prediction_inputs)))
                                                                                            
    results, preds_list = utils.evaluate_bert_model(prediction_dataloader, BATCH_SIZE, model, tokenizer, index2label, 
                                                        PAD_TOKEN_LABEL_ID, full_report=True, prefix="Test Set")
    logging.info("  Test Loss: {0:.2f}".format(results['loss']))
    logging.info("  Precision: {0:.2f} || Recall: {1:.2f} || F1: {2:.2f}".format(results['precision']*100, results['recall']*100, results['f1']*100))

    with open(OUTPUTS_PATH, "w") as fout:
        with open(INPUTS_PATH, "w") as fin:
            for sent, pred in preds_list:
                fin.write(" ".join(sent)+"\n")
                fout.write(" ".join(pred)+"\n")

else:
    # https://huggingface.co/transformers/main_classes/pipelines.html#transformers.TokenClassificationPipeline
    logging.info('Predicting labels for {:,} test sentences...'.format(len(test_data)))
    if not USE_CUDA: GPU_IX = -1
    nlp = pipeline('token-classification', model=model, tokenizer=tokenizer, device=GPU_IX)
    nlp.ignore_labels = []
    with open(OUTPUTS_PATH, "w") as fout:
        with open(INPUTS_PATH, "w") as fin:
            for seq_ix, seq in enumerate(test_data):
                sentence = " ".join(seq)
                predicted_labels = []
                output_obj = nlp(sentence)
                # [print(o) for o in output_obj]
                for tok in output_obj:
                    if '##' not in tok['word']:
                        predicted_labels.append(tok['entity'])
                logging.info(f"\n----- {seq_ix+1} -----\n{seq}\nPRED:{predicted_labels}")
                fin.write(sentence+"\n")
                fout.write(" ".join(predicted_labels)+"\n")

INFO:root:Predicting labels for 17 test sentences...
INFO:bert_utils_srl:***** Running evaluation Test Set *****
INFO:bert_utils_srl:  Batch size = 4
INFO:bert_utils_srl:{0: 'O', 1: 'B-ARG0', 2: 'B-V', 3: 'B-ARG1', 4: 'B-ARGM-LOC', 5: 'B-ARGM-MOD', 6: 'B-ARGM-GOL', 7: 'B-ARGM-TMP', 8: 'B-ARG2', 9: 'B-ARGM-ADV', 10: 'B-ARGM-MNR', 11: 'B-ARGM-LVB', 12: 'B-ARGM-PRR', 13: 'B-ARG4', 14: 'B-ARGM-NEG', 15: 'B-ARGM-EXT', 16: 'B-R-ARG0', 17: 'B-ARGM-ADJ', 18: 'B-R-ARG1', 19: 'B-C-V', 20: 'B-ARG3', 21: 'B-ARGM-COM', 22: 'B-ARGM-DIS', 23: 'B-ARGM-CAU'}
INFO:bert_utils_srl:
----- 1 -----
['What', 'if', 'Google', 'Morphed', 'Into', 'GoogleOS', '?']

GOLD: ['O', 'O', 'B-ARG1', 'B-V', 'O', 'B-ARG2', 'O']
PRED:['O', 'O', 'O', 'O', 'O', 'O', 'O']

INFO:bert_utils_srl:
----- 2 -----
['What', 'if', 'Google', 'expanded', 'on', 'its', 'search', '-', 'engine', '(', 'and', 'now', 'e-mail', ')', 'wares', 'into', 'a', 'full', '-', 'fledged', 'operating', 'system', '?']

GOLD: ['O', 'O', 'B-ARG0', 'B-V', 'O', '

/opt/anaconda3/envs/bert4ner/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/envs/bert4ner/lib/python3.8/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
